# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install git+https://github.com/rwightman/pytorch-image-models.git 
#!pip install timm==0.4.5

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import datasets, transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import csv
import os
import cv2
import sys
sys.path.append(r'../input/cvprvit/')
import timm
import albumentations as albu
from albumentations.pytorch import ToTensorV2

device = ('cuda' if torch.cuda.is_available() else 'cpu')




csvFile = open(r"./submission.csv", "w")  # 创建csv文件
writer = csv.writer(csvFile, lineterminator="\n")  # 创建写的对象
# 先写入columns_name
writer.writerow(["image", "labels"])  # 写入列的名称

# 读入图片
test_root = r"../input/plant-pathology-2021-fgvc8/test_images/"
img_test = os.listdir(test_root)


model = torch.load(r'../input/cvprvit/vit_large_patch16_384_epoch_6.pth')
model.to(device)
#print(model)

for i in range(len(img_test)):
    img = cv2.imread(test_root + img_test[i])
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
#     img = Image.open(test_root + img_test[i])
#     img = img.convert('RGB')
#     transform = torchvision.transforms.Compose(
#         [
#             torchvision.transforms.Resize(384),
#             torchvision.transforms.CenterCrop(384),
#             torchvision.transforms.ToTensor(),
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#         ]
#     )
    transform = albu.Compose([
    albu.Resize(height=384, width=384, p=1.0),
    albu.Normalize(
        mean=[0.5000, 0.5000, 0.5000],
        std=[0.5000, 0.5000, 0.5000],),
    ToTensorV2(),
    ])
    input = transform(image=img)['image']
    #input = transform(img)
    input = input.unsqueeze(0)  # 这里经过转换后输出的input格式是[C,H,W],网络输入还需要增加一维批量大小B
    # 增加一维，输出的img格式为[1,C,H,W]
    input = input.cuda()
    with torch.no_grad():
        score = model(input)  # 将图片输入网络得到输出
    
        score = torch.nn.functional.softmax(score, dim=1)
        print(score)
        prediction = torch.max(score, 1)[1]
        pred = int(prediction[0])
        rlabel = ''
        if pred == 0:
            rlabel = "complex"
        if pred == 1:
            rlabel = "rust"
        if pred == 2:
            rlabel = "scab"
        if pred == 3:
            rlabel = "healthy"
        if pred == 4:
            rlabel = "powdery_mildew"
        if pred == 5:
            rlabel = "frog_eye_leaf_spot"
        print(rlabel)
        writer.writerow([img_test[i], rlabel])

csvFile.close()